In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
filefolder = r'C:\Users\johan\OneDrive\Marcobre\1.0 PROJECTS\01. Crear budget de mantenimiento\hr2csv'
filename = r'5._Carga de hoja de ruta - GRUPO 12 (03.11) UBTC.xlsx'
mergeHRname = r'HR\csv\mergeHR.csv'
a = r'HR'
EQ2GHRname = r'Hoja Resumen- CARGA A SAP.xlsx'

In [3]:
HRrute = os.path.join(filefolder,a,filename)
mergeHRrute = os.path.join(filefolder,mergeHRname)
EQ2GHRrute = os.path.join(filefolder,EQ2GHRname)

In [484]:
flag = 0
grupo1 = 12
grupo2 = 12

In [485]:
#db1 = pd.read_excel(HRrute,sheet_name=flag, skiprows = 6, header = None)
db = pd.read_csv(r'C:\Users\johan\OneDrive\Marcobre\1.0 PROJECTS\01. Crear budget de mantenimiento\hr2csv\allHRdata.csv')
eq2ghr = pd.read_excel(EQ2GHRrute,sheet_name=0, header=0)

In [486]:
eq2ghr.drop(columns=['OBSERVACIONES','PLAN DE MANTENIMIENTO','HOJA DE RUTA TIPO EQUIPO','HOJA DE RUTA INSTRUCCIÓN'],inplace=True)

In [487]:
eq2ghr.columns = ['TAG','DescripcionEquipo','GrupoTS','GrupoHR','EqSAP','PuntoMedida']
#eq2ghr = eq2ghr.loc[(eq2ghr.loc[:,'GrupoTS']== grupo1) | (eq2ghr.loc[:,'GrupoTS']== grupo2),:]
#eq2ghr.loc[(eq2ghr.loc[:,'EqSAP']== 'UBT','EqSAP')] = 0
#eq2ghr.loc[:,'EqSAP'] = eq2ghr.loc[:,'EqSAP'].astype(float)

In [488]:
db.columns = db.columns.astype(int)

In [489]:
# Hojas de ruta
HR = db.loc[db.iloc[:,0] == 1,:]
HR.drop(labels=[11,12,13,14,15,16,17], axis = 1, inplace = True)
HRcolumnsname = [0,'GrupoHREQ','Contador','HRDescripcion','HRPuesto','Centro','Utilizacion','GPlanificacion', 'EstadoGlobal','EstadoInstalacion','Estrategia']
HR.columns = HRcolumnsname
HR.loc[:,'HRContador'] = HR['GrupoHREQ'].astype(str) + HR['Contador']
# Add maintenance estrategy of each actiivitie, and type of equipment
HR.loc[:,'EstrategiaMTTO'] = HR.loc[:,'HRDescripcion'].str.split('_',expand = True)[0] 
HR.loc[:,'ClaseEq'] = HR.loc[:,'HRDescripcion'].str.split('_',expand = True)[2]

In [493]:
EQlist = HR.drop_duplicates(subset = ['GrupoHREQ'])
#EQlist.drop(labels = [0,2,3,4,5,6,7,8,9,10], axis=1, inplace= True)
EQlist.reset_index(drop= True,inplace=True)

In [494]:
# Posiciones de las Hojas de Ruta
PHR = db.loc[db.iloc[:,0] == 2,:]
PHR.drop(columns = [10,11,12,13,14,15,16,17],inplace = True)
PHRcolumnsname = [0,'GrupoHREQ','pos','Puesto','ClaseOT','Actividades','HH','H','ACant','AHoras']
PHR.columns = PHRcolumnsname

In [495]:
# Actividad detallada de laPosiciones de las Hojas de Ruta
AHR = db.loc[db.iloc[:,0] == 3,:]

In [496]:
%%time
# Conectar Posiciones de HR con la HR a traves de HRContador del HR al PHR
nm1=0
#HR.tail(1).index.values[0] = last index of HR dataframe
for m in PHR.index:
    for n in HR.index:
        if nm1<=m and m<=n:
            PHR.loc[m,'HRContador'] = HR.loc[nm1,'HRContador']
            break
        elif m>=HR.tail(1).index.values[0]:
            PHR.loc[m,'HRContador'] = HR.loc[HR.tail(1).index.values[0],'HRContador']
            break
        nm1 = n
PHR.loc[:,'posHRContador'] = PHR.loc[:,'HRContador'] + PHR.loc[:,'pos'].astype(str)

Wall time: 27min 32s


In [497]:
# Materiales de Hojas de Ruta
MHR = db.loc[db.iloc[:,0] == 4,:]
MHR = MHR.drop(columns = [4,5,6,7,8,9,10,11,12,13,14,15,16,17])
MHRcolumnsname = [0,'GrupoHREQ','Material','MCantidad']
MHR.columns = MHRcolumnsname

In [499]:
%%time
# Conectar Posiciones de HR con la HR a traves de posHRContador del PHR al MHR
nm1=0
#HR.tail(1).index.values[0] = last index of HR dataframe
for m in MHR.index:
    for n in PHR.index:
        if nm1<=m and m<=n:
            MHR.loc[m,'posHRContador'] = PHR.loc[nm1,'posHRContador']
            MHR.loc[m,'HRContador'] = PHR.loc[nm1,'HRContador']
            break
        elif m>=PHR.tail(1).index.values[0]:
            MHR.loc[m,'posHRContador'] = PHR.loc[PHR.tail(1).index.values[0],'posHRContador']
            MHR.loc[m,'HRContador'] = PHR.loc[PHR.tail(1).index.values[0],'HRContador']
            break
        nm1 = n
#PHR.loc[:,'posHRContador'] = PHR.loc[:,'HRContador'] + PHR.loc[:,'pos'].astype(str)

Wall time: 8min 57s


In [500]:
# Materiales de Estrategia de Ruta
EPHR = db.loc[db.iloc[:,0] == 5,:]
EPHR.drop(columns = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],inplace = True)
EPHR.columns = [0,'GrupoHREQ','Frecuencia']

In [501]:
%%time
# Conectar Estrategias de HR con la posicion de la HR
nm1=0
#HR.tail(1).index.values[0] = last index of HR dataframe
for m in EPHR.index:
    for n in PHR.index:
        if nm1<=m and m<=n:
            EPHR.loc[m,'posHRContador'] = PHR.loc[nm1,'posHRContador']
            #EPHR.loc[m,'HRContador'] = PHR.loc[nm1,'HRContador']
            break
        elif m>=PHR.tail(1).index.values[0]:
            EPHR.loc[m,'posHRContador'] = PHR.loc[PHR.tail(1).index.values[0],'posHRContador']
            #EPHR.loc[m,'HRContador'] = PHR.loc[HR.tail(1).index.values[0],'HRContador']
            break
        nm1 = n
#PHR.loc[:,'posHRContador'] = PHR.loc[:,'HRContador'] + PHR.loc[:,'pos'].astype(str)

Wall time: 48min 45s


In [502]:
PHRE = pd.merge(PHR.loc[:,['pos','Actividades','Puesto','ClaseOT','ACant','AHoras','HRContador','posHRContador']],EPHR.loc[:,['Frecuencia','posHRContador']],on = 'posHRContador',how = 'left')

In [503]:
posHRwithMHR = pd.merge(PHRE.loc[:,['pos','Actividades','Puesto','ClaseOT','ACant','AHoras','HRContador','posHRContador','Frecuencia']],MHR.loc[:,['posHRContador','Material','MCantidad']],on='posHRContador',how='left')

In [504]:
allDutiesMat = pd.merge(HR.loc[:,['GrupoHREQ','Contador','HRPuesto','EstrategiaMTTO','HRDescripcion','HRContador','Estrategia']],posHRwithMHR,on='HRContador',how='left')

In [ ]:
#allDutiesMat = pd.merge(HR.loc[:,['GrupoHREQ','Contador','HRPuesto','EstrategiaMTTO','HRDescripcion','HRContador','Estrategia']],PHRE.loc[:,['pos','Actividades','Puesto','ClaseOT','ACant','AHoras','HRContador','posHRContador','Frecuencia']],on='HRContador',how='left')

In [ ]:
#allDutiesMat.to_csv(mergeHRrute, encoding = 'utf-8-sig')

In [505]:
#if flag == 1:    
#    PruebaEQ = pd.merge(eq2ghr.loc[:,['TAG', 'DescripcionEquipo', 'GrupoTS', 'GrupoHR', 'EqSAP', #'PuntoMedida']],allDutiesMat,left_on='EqSAP',right_on='GrupoHREQ',how='right')
#    PruebaEQ.to_csv(mergeHRrute, encoding = 'utf-8-sig')

In [506]:
#if flag == 0:    
#    PruebaINS = pd.merge(eq2ghr.loc[:,['TAG', 'DescripcionEquipo', 'GrupoTS', 'GrupoHR', 'EqSAP', #'PuntoMedida']],allDutiesMat,left_on='GrupoHR',right_on='GrupoHREQ',how='right')
#    PruebaINS.to_csv(mergeHRrute, encoding = 'utf-8-sig')

In [527]:
allDutiesMat.loc[:,'EstrategiaMTTO'].unique()

array(['COND', 'INSP', 'PREV', 'LUBR', 'PRED', nan], dtype=object)

In [525]:
allDutiesMat.loc[allDutiesMat.loc[:,'EstrategiaMTTO'] == allDutiesMat.loc[:,'EstrategiaMTTO'].unique()[5] ,:]

,GrupoHREQ,Contador,HRPuesto,EstrategiaMTTO,HRDescripcion,HRContador,Estrategia,pos,Actividades,Puesto,ClaseOT,ACant,AHoras,posHRContador,Frecuencia,Material,MCantidad


In [ ]:

PruebaEQ = pd.merge(eq2ghr.loc[:,['TAG', 'DescripcionEquipo', 'GrupoTS', 'GrupoHR', 'EqSAP', 'PuntoMedida']],allDutiesMat,left_on='EqSAP',right_on='GrupoHREQ',how='right')

In [ ]:
PruebaINS = pd.merge(eq2ghr.loc[:,['TAG', 'DescripcionEquipo', 'GrupoTS', 'GrupoHR', 'EqSAP', 'PuntoMedida']],allDutiesMat,left_on='GrupoHR',right_on='GrupoHREQ',how='right')